# api

> High-level API and entry point for beetroot

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *
from IPython.display import display, Markdown
import json
import os
from pathlib import Path
import re
import tempfile


In [ ]:
#| hide
#| export
from typing import Dict, Iterable, List, Protocol, Sequence, TypeVar

In [ ]:
# | export
class SourceHandler(Protocol):
    """High-level API for handling cell source"""

    def handle_markdown(self, lines: Sequence[str]):
        """Handle the source lines for a markdown cell
        
        Params:
            lines: The lines of source text
        """
        pass

    def handle_python_source(self, lines: Sequence[str]) -> bool:
        """Handle the source lines for a python code cell
        
        Params:
            lines: The lines of source text

        Returns:
            bool: Indicates whether the output should be handled for
                  this cell.
        """
        return False

In [ ]:
# | exporti
TOutputResult = TypeVar('TOutputResult', covariant=True)

In [ ]:
# | export
class OutputHandler(Protocol[TOutputResult]):
    """High-level API for handling cell outputs"""
    def handle_output(self, output: Dict) -> TOutputResult:
        """Handle a single cell output
        
        Params:
            output: a Dict representing a JSON output element as 
                    defined in the [notebook file format](https://nbformat.readthedocs.io/en/latest/format_description.html#code-cell-outputs)

        Returns:
            TOutputResult: a result whose type is defined by the implementation 
                           (typically a callable that completes any 
                           asynchronous processing required)                    
        """
        pass

In [ ]:
# | export
def export_notebook(
    nb_json: Dict,
    source_handler: SourceHandler,
    output_handler: OutputHandler[TOutputResult],
) -> Iterable[TOutputResult]:
    results: List[TOutputResult] = []
    
    for cell in nb_json['cells']:
        if cell['cell_type'] == 'markdown':
            source_handler.handle_markdown(cell['source'])
        elif cell['cell_type'] == 'code':
            should_show_output = source_handler.handle_python_source(cell['source'])

            if not should_show_output:
                continue

            for output in cell['outputs']:
                result = output_handler.handle_output(output)
                results.append(result)


    return results

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()